In [5]:
# IMPORTS

import numpy as np
from pathlib import Path
import glob2
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
from src.models.unet import UNet

from src.visualization.visualization_fct import mask_to_rgb

import cv2
from torchvision.transforms import Normalize

# CONSTANTS

path = '../data/to_infer/'
path_result=path+'results/'

In [7]:
# read image in target folder
all_paths = [ Path(p).absolute() for p in glob2.glob(path + '*.png') ]
all_paths = all_paths+[ Path(p).absolute() for p in glob2.glob(path + '*.jpg') ]
print('Number of files:', len(all_paths))
print(all_paths)

Number of files: 5
[PosixPath('/work3/s212803/dl_kitkars/notebooks/../data/to_infer/1_a.png'), PosixPath('/work3/s212803/dl_kitkars/notebooks/../data/to_infer/0_a.png'), PosixPath('/work3/s212803/dl_kitkars/notebooks/../data/to_infer/33_a.png'), PosixPath('/work3/s212803/dl_kitkars/notebooks/../data/to_infer/3_a.png'), PosixPath('/work3/s212803/dl_kitkars/notebooks/../data/to_infer/KitKars.jpg')]


In [8]:
# load model

name_model = 'expFinal_epoch500'

device = torch.device('cpu')
model = UNet(n_channels=3, n_classes=9)
model.load_state_dict(torch.load(f'../models/unet_finetuned_{name_model}.pt', map_location=torch.device('cpu')))
model = model.to(device)

In [9]:
for aPath in tqdm(all_paths):
    print(aPath)
    rgb_img_raw = cv2.imread(str(aPath))
    rgb_img_raw = rgb_img_raw[:, :, [2, 1, 0]]
    rgb_img = np.transpose(rgb_img_raw, (2, 0, 1))

    rgb_img = torch.Tensor(rgb_img).type(torch.float)
    rgb_img = Normalize((127.5,127.5,127.5), (127.5,127.5,127.5)).forward(rgb_img)
    output = model(rgb_img.view([1]+list(np.array(rgb_img.size()))))[0]
    predictions = np.argmax(output.detach().numpy(), axis=0)
    alpha_array = np.argwhere(predictions==0)
    predicted_mask_img = mask_to_rgb(predictions)

    for aPosition in alpha_array:
        predicted_mask_img[aPosition[0] ,aPosition[1], :] = rgb_img_raw[aPosition[0] ,aPosition[1], :]
    
    # save img
    predicted_mask_img_bgr = predicted_mask_img[:, :, [2, 1, 0]]
    cv2.imwrite(path_result+aPath.stem+'.png', predicted_mask_img_bgr)

  0%|          | 0/5 [00:00<?, ?it/s]

/work3/s212803/dl_kitkars/notebooks/../data/to_infer/1_a.png


 20%|██        | 1/5 [01:08<04:33, 68.26s/it]

/work3/s212803/dl_kitkars/notebooks/../data/to_infer/0_a.png


 40%|████      | 2/5 [02:14<03:22, 67.34s/it]

/work3/s212803/dl_kitkars/notebooks/../data/to_infer/33_a.png


 60%|██████    | 3/5 [03:20<02:12, 66.50s/it]

/work3/s212803/dl_kitkars/notebooks/../data/to_infer/3_a.png


 80%|████████  | 4/5 [04:26<01:06, 66.19s/it]

/work3/s212803/dl_kitkars/notebooks/../data/to_infer/KitKars.jpg


In [ ]:
#viz
plt.title('Our prediction')
plt.imshow(predicted_mask_img)
plt.axis(False)
plt.show()